## 1. Syntactic Patterns for Technical Terms ##

In [2]:
import nltk
from nltk.corpus import brown

As seen in the Manning and Schuetze chapter, there is a well-known part-of-speech 
based pattern defined by Justeson and Katz for identifying simple noun phrases 
that often works well for pulling out keyphrases.

 Technical Term  T = (A | N)+ (N | C)  | N

Below, write a function to  define a chunker using the RegexpParser as illustrated in the NLTK book Chapter 7 section 2.3 *Chunking with Regular Expressions*.  You'll need to revise the grammar rules shown there to match the pattern shown above.  You can be liberal with your definition of what is meant by *N* here.  Also, C refers to cardinal number, which is CD in the brown corpus.



In [34]:
# Technical Term T = (A | N)+ (N | C) | N
grammar = r"""
  T: {(<JJ.|NN.*>)+(<NN.*|CD>)|<NN.*>}   # chunk technical term
"""
cp = nltk.RegexpParser(grammar)
sentence = [("Rapunzel", "NNP"), ("let", "VBD"), ("down", "RP"), 
                 ("her", "PP$"), ("long", "JJ"), ("golden", "JJ"), ("hair", "NN")]
print(cp.parse(sentence))

(S
  (T Rapunzel/NNP)
  let/VBD
  down/RP
  her/PP$
  long/JJ
  golden/JJ
  (T hair/NN))


Below, write a function to call the chunker, run it on some sentences, and then print out the results for  those sentences.

For uniformity, please run it on sentences 100 through 104 from the full tagged brown corpus.

 

In [35]:
brown_tagged = brown.tagged_sents()
for sent in brown_tagged[100:104]:
    print(cp.parse(sent))

(S
  Daniel/NP
  personally/RB
  led/VBD
  the/AT
  (T fight/NN)
  for/IN
  the/AT
  (T measure/NN)
  ,/,
  which/WDT
  he/PPS
  had/HVD
  watered/VBN
  down/RP
  considerably/RB
  since/IN
  its/PP$
  (T rejection/NN)
  by/IN
  two/CD
  previous/JJ
  (T Legislatures/NNS-TL)
  ,/,
  in/IN
  a/AT
  public/JJ
  (T hearing/NN)
  before/IN
  the/AT
  (T House/NN-TL Committee/NN-TL)
  on/IN-TL
  (T Revenue/NN-TL)
  and/CC-TL
  (T Taxation/NN-TL)
  ./.)
(S
  Under/IN
  (T committee/NN rules/NNS)
  ,/,
  it/PPS
  went/VBD
  automatically/RB
  to/IN
  a/AT
  (T subcommittee/NN)
  for/IN
  one/CD
  (T week/NN)
  ./.)
(S
  But/CC
  (T questions/NNS)
  with/IN
  which/WDT
  (T committee/NN members/NNS)
  taunted/VBD
  (T bankers/NNS)
  appearing/VBG
  as/CS
  (T witnesses/NNS)
  left/VBD
  little/AP
  (T doubt/NN)
  that/CS
  they/PPSS
  will/MD
  recommend/VB
  (T passage/NN)
  of/IN
  it/PPO
  ./.)
(S
  Daniel/NP
  termed/VBD
  ``/``
  extremely/RB
  conservative/JJ
  ''/''
  his/PP$
  (T estim


Then extract out the phrases themselves on sentences 100 through 160 using the subtree extraction technique shown in the 
*Exploring Text Corpora* category.  

In [36]:
for sent in brown_tagged[100:160]:
    tree = cp.parse(sent)
    for subtree in tree.subtrees():
        if subtree.label() == 'T': print(subtree)

(T fight/NN)
(T measure/NN)
(T rejection/NN)
(T Legislatures/NNS-TL)
(T hearing/NN)
(T House/NN-TL Committee/NN-TL)
(T Revenue/NN-TL)
(T Taxation/NN-TL)
(T committee/NN rules/NNS)
(T subcommittee/NN)
(T week/NN)
(T questions/NNS)
(T committee/NN members/NNS)
(T bankers/NNS)
(T witnesses/NNS)
(T doubt/NN)
(T passage/NN)
(T estimate/NN)
(T dollars/NNS)
(T deficit/NN)
(T dollars/NNS)
(T end/NN)
(T year/NN)
(T committee/NN)
(T measure/NN)
(T means/NNS)
(T escheat/NN law/NN)
(T books/NNS)
(T republic/NN)
(T state/NN)
(T bank/NN accounts/NNS)
(T stocks/NNS)
(T property/NN)
(T persons/NNS)
(T years/NNS)
(T bill/NN)
(T banks/NNS)
(T insurance/NN firms/NNS)
(T pipeline/NN companies/NNS)
(T corporations/NNS)
(T property/NN)
(T state/NN treasurer/NN)
(T escheat/NN law/NN)
(T property/NN)
(T lawyer/NN)
(T Bankers/NNS-TL Association/NN-TL)
(T opposition/NN keynote/NN)
(T banks/NNS)
(T obligations/NNS)
(T depositors/NNS)
(T confidence/NN)
(T bank/NN customers/NNS)
(T confidence/NN)
(T banks/NNS)
(T 

## 2. Identify Proper Nouns ##
For this next task, write a new version of the chunker, but this time change it in two ways:
 1. Make it recognize proper nouns
 2. Make it work on your personal text collection which means that you need to run a tagger over your personal text collection.

Note that the second requirements means that you need to run a tagger over your personal text collection before you design the proper noun recognizer.  You can use a pre-trained tagger or train your own on one of the existing tagged collections (brown, conll, or treebank)



In [4]:
import re
tal = open("../tal_stories/tal_text.txt", "r")
tal_text = tal.read()

tal.close()
# remove tags

tal_text[:100]


'<EPISODE NUMBER:496> <EPISODE NAME:When Patents Attack... Part Two!> <HOST:IRA GLASS>  OK. I listen '

In [ ]:
sent_tokenizer = nltk.data.load('tokenizers/punkt/english.pickle')
raw_sents = sent_tokenizer.tokenize(tal_text)
tal_sents = [nltk.word_tokenize(word) for word in raw_sents]
tal_sents[100:103]

**Tagger:** Your code for optionally training tagger, and for definitely running tagger on your personal collection goes here:

In [54]:
def create_data_sets(sentences):
    size = int(len(sentences) * 0.9)
    train_sents = sentences[:size]
    test_sents = sentences[size:]
    return train_sents, test_sents

def build_backoff_tagger (train_sents):
    t0 = nltk.DefaultTagger('NN')
    t1 = nltk.UnigramTagger(train_sents, backoff=t0)
    t2 = nltk.BigramTagger(train_sents, backoff=t1)
    t3 = nltk.TrigramTagger(train_sents, backoff=t2)
    return t3

train_sents, test_sents = create_data_sets(tal_sents)
ngram_tagger = build_backoff_tagger(brown_tagged)
tal_tagged = []
for sent in tal_sents:
    tal_tagged.append(ngram_tagger.tag(sent)) 
tal_tagged[1:10]

[[('I', 'PPSS'), ('listen', 'VB'), ('to', 'TO'), ('a', 'AT'), ('lot', 'NN'), ('of', 'IN'), ('podcasts', 'NN'), ('.', '.')], [('I', 'PPSS'), ('host', 'NN'), ('a', 'AT'), ('radio', 'NN'), ('show', 'NN'), ('that', 'WPS'), ('is', 'BEZ'), ('distributed', 'VBN'), ('as', 'CS'), ('a', 'AT'), ('podcast', 'NN'), ('.', '.')], [('So', 'RB'), ('I', 'PPSS'), ('had', 'HVD'), ('a', 'AT'), ('special', 'JJ'), ('interest', 'NN'), ('in', 'IN'), ('some', 'DTI'), ('interviews', 'NNS'), ('that', 'WPS'), ('one', 'CD'), ('of', 'IN'), ('my', 'PP$'), ('colleagues', 'NNS'), (',', ','), ('Zoe', 'NP'), ('Chace', 'NN'), (',', ','), ('did', 'DOD'), ('recently', 'RB'), ('.', '.')], [('She', 'PPS'), ("'s", 'NN'), ('one', 'CD'), ('of', 'IN'), ('the', 'AT'), ('reporters', 'NNS'), ('for', 'IN'), ('Planet', 'NN'), ('Money', 'NN'), (',', ','), ('which', 'WDT'), ('itself', 'PPL'), ('is', 'BEZ'), ('a', 'AT'), ('podcast', 'NN'), ('.', '.')], [('Zoe', 'NP'), ('was', 'BEDZ'), ('interviewing', 'VBG'), ('these', 'DTS'), ('two', 'C

**Chunker:** Code for the proper noun chunker goes here:

In [64]:
# Technical Term T = (A | N)+ (N | C) | N
grammar = r"""
  T: {(<JJ.|NP.*>)+(<NP.*|CD>)|<NP.*>}   # chunk technical term
"""
cp = nltk.RegexpParser(grammar)

**Test the Chunker:** Test your proper noun recognizer on a lot of sentences to see how well it is working.  You might want to add prepositions in order to improve your results.  


In [75]:
nps = []
for sent in tal_tagged[1000:1600]:
    tree = cp.parse(sent)
    for subtree in tree.subtrees():
        if subtree.label() == 'T': nps.append(str(subtree))
            
print(set(nps))

{'(T Brian/NP Reed/NP)', '(T William/NP)', '(T Miami/NP)', '(T Dave/NP Mitchell/NP)', '(T Jorge/NP)', '(T Oscar/NP)', '(T David/NP)', '(T Dave/NP)', '(T Miami/NP-TL)', '(T Patricia/NP)', '(T Lisa/NP)', '(T Julie/NP Snyder/NP)', '(T Chicago/NP)', '(T Ben/NP Calhoun/NP)', '(T Colombian/NP)', '(T Miguel/NP)', '(T Colombia/NP)', '(T California/NP)', '(T Lemon/NP)', '(T Florida/NP)', '(T July/NP 15/CD)', '(T Mac/NP)', '(T Chris/NP)', '(T American/NP)', '(T China/NP)', '(T Virginia/NP)', '(T Columbia/NP)', '(T Mike/NP)', '(T California/NP-TL)', '(T Alex/NP Blumberg/NP)', '(T Nancy/NP)', '(T Jersey/NP)', '(T Coke/NP)', '(T Karen/NP Lowe/NP)', '(T Cubans/NPS)', '(T Congress/NP)', '(T Macintosh/NP)', '(T England/NP)', '(T Sarah/NP Koenig/NP)', '(T Rob/NP Geddes/NP)', '(T Jonathan/NP)', '(T Bell/NP-TL)', '(T Matt/NP)', '(T Mario/NP)', '(T God/NP)', '(T Castillo/NP)', '(T Ira/NP)', '(T Anton/NP)', '(T David/NP Mitchell/NP)', '(T Columbia/NP-TL)', '(T Julian/NP)', '(T Pablo/NP)'}


**FreqDist Results:** After you have your proper noun recognizer working to your satisfaction, below  run it over your entire collection, feed the results into a FreqDist, and then print out the top 20 proper nouns by frequency.  That code goes here, along with the output:


In [79]:
nps = []
for sent in tal_tagged:
    tree = cp.parse(sent)
    for subtree in tree.subtrees():
        if subtree.label() == 'T': nps.append(str(subtree))

fd = nltk.FreqDist(nps)
fd.most_common(10)

[('(T Jorge/NP)', 84), ('(T Anthony/NP)', 80), ('(T Jon/NP)', 78), ('(T Chris/NP Crawford/NP)', 77), ('(T Sam/NP)', 63), ('(T Joe/NP)', 50), ('(T Adrian/NP)', 42), ('(T Steve/NP)', 35), ('(T American/NP)', 34), ('(T Miguel/NP)', 34)]